## **Collect Modis Data from GEE**
### 1. Imports and Configuration

In [1]:
import ee
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import rasterio

pd.options.mode.chained_assignment = None
ee.Authenticate()
ee.Initialize(project='geototh-learning1')

### 2. Retrieve data from GEE, filter by date and location

In [15]:
def get_gdf_from_GEE(collection,start_date,end_date,bbox):
    """Get a GDF of data from GEE

    Arguments
    collection : the GEE collection to pull from
    start_date : the first date of of images to retrieve
    end_date   : the last date of of images to retrieve
    bbox       : Boundaries of the AOI as ee.Geometry object

    Returns
    IC_gdf : GeoDataFrame of the entire image collection
    """
    
    img_collection = ee.ImageCollection(collection)\
        .filterDate(start_date, end_date)
    
    # Cast ImageCollection to xarray.DataSet:
    ds = xr.open_dataset(img_collection,
                         engine='ee',
                         crs='EPSG:4326',
                         scale=.01,
                         geometry=bbox)
    del(img_collection) # deallocate this memory ASAP
    
    # Cast DataSet to GeoDataFrame:
    df = ds.to_dataframe()
    del(ds) # deallocate this memory ASAP
    df = df.reset_index(level = ['lon','lat','time'])
    IC_gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
    
    return(IC_gdf)


collection = "MODIS/061/MOD09GA"
co_bbox = ee.Geometry.BBox(-109.059196,36.992751,-102.042126,41.001982)


start_date = "2023-08-01"
end_date = "2023-09-01"
IC_gdf_202308 = get_gdf_from_GEE(collection,start_date,end_date,co_bbox)


start_date = "2024-03-01"
end_date = "2024-04-01"
IC_gdf_202403 = get_gdf_from_GEE(collection,start_date,end_date,co_bbox)

print("done")

done


### 3. Select bands and export to local directory

In [16]:
def export_IC(IC,bands,fldr,f_name):
    """Writes IC_gdf to parquet file partitioned on 'time'

    Arguments
    IC     : ImageCollection to be written (gdf)
    bands  : bands to include in export (list)
    fldr   : directory to export data (string)
    f_name : file name (string)

    Returns
    None
    """
    IC = IC[bands]
    
    #export as parquet
    IC.to_parquet(
        path=fldr+f_name,
    ) # want to add partition functionality to this later
    
    return(None)


bands = ['time','sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04',
           'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07','geometry','lon','lat']
fldr = "C:\\Users\\Chris\\co_cloudcover\\data\\raw\\"


f_name = "ImageCollection_202308.parquet"
export_IC(IC_gdf_202308,bands,fldr,f_name)

f_name = "ImageCollection_202403.parquet"
export_IC(IC_gdf_202403,bands,fldr,f_name)

print("done")

done


### **Export GeoDataFrame as daily GeoTiffs**
These tiffs are loaded into QGIS for label selection
#### 1. Define functions for gtiff export

In [5]:
from rasterio.transform import Affine
import math

def calculate_affine(coeffs):
    """Calculate Affine transformation for rasterio write
    
    Args described here:
    https://trac.osgeo.org/postgis/wiki/DevWikiAffineParameters"""
    sx,sy,tx,ty = coeffs['sx'],coeffs['sy'],coeffs['tx'],coeffs['ty']
    θ,kx,ky = coeffs['θ'],coeffs['kx'],coeffs['ky']
    
    o11 = sx * ((1 + kx*ky) * math.cos(θ) + ky*math.sin(θ) )
    o12 = sx * ( kx*math.cos(θ) + math.sin(θ) )
    o21 = -sy * ( -(1 + kx*ky) * math.sin(θ) + ky*math.cos(θ) )
    o22 = sy * ( -kx*math.sin(θ) + math.cos(θ) )

    return(Affine(o11,o12,tx,o21,o22,ty))

def rbgArray_to_RGBgeotiff(rgb,f_name,directory,profile):
    """Writes 3 dimensional array as 3-band tiff
    
    Arguments
    rgb       : 3xMxN ndarray of channels
    f_name    : name of image
    directory : location to export image
    profile   : parameters for rasterio write
    
    Returns
    None
    """
    with rasterio.open(directory + f_name,'w',**profile) as dst:
        dst.write(rgb)

    return(None)
    

def make_RGBarray(gdf,img_date,rgb_bands):
    """constructs 3-band RGB arrays for selected date in gdf

    Arguments
    gdf       : ImageCollection as GeoDataFrame
    img_date  : date to extract from gdf
    rgb_bands : dict of bands to use for RGB image

    Returns
    rgb   : 3xMxN ndarray of channels
    dim_x : len of x axis
    dim_y : len of y axis
    """
    
    #select data by date:
    date_gdf = gdf[gdf['time']==img_date]
    
    # Select Bands:
    r_band = rgb_bands["red"]
    g_band = rgb_bands["green"]
    b_band = rgb_bands["blue"]
    rgb_gdf = date_gdf[[r_band,g_band,b_band,"geometry"]]
    
    # Extract Geometry:
    rgb_gdf['x'],rgb_gdf['y'] = rgb_gdf.geometry.x,rgb_gdf.geometry.y
    dim_x,dim_y = len(set(rgb_gdf.x)),len(set(rgb_gdf.y))

    # Construct channel arrays
    r_array = rgb_gdf[r_band].to_numpy().reshape((dim_x,dim_y))
    g_array = rgb_gdf[g_band].to_numpy().reshape((dim_x,dim_y))
    b_array = rgb_gdf[b_band].to_numpy().reshape((dim_x,dim_y))

    #Construct RGB Array
    rgb = np.stack((r_array,g_array,b_array))
    
    return(rgb,dim_x,dim_y)

#### 2.Write snow-enhanced gtiffs
These are RGB blends using Red (band 1), NIR (band 5), and SWIR (band 7), respectively

In [17]:
IC_gdf = IC_gdf_202308

rgb_bands = {
    "profile_name" : "snow_enhanced",
    "red":"sur_refl_b01",
    "green":"sur_refl_b05",
    "blue":"sur_refl_b07",
    }

img_dates = list(set(IC_gdf["time"]))
for img_date in img_dates:
    img_date = img_date.strftime("%Y-%m-%d")
    
    rgb,dim_x,dim_y = make_RGBarray(IC_gdf,img_date,rgb_bands)
    
    transform_coeffs = {
        'sx':abs((min(IC_gdf['lon']) - max(IC_gdf['lon']))/dim_y)  ,# scale factor in x direction
        'sy':abs((min(IC_gdf['lat']) - max(IC_gdf['lat']))/dim_x)  ,# scale factor in y direction
        'tx':min(IC_gdf['lon'])  ,# offset in x direction
        'ty':min(IC_gdf['lat'])  ,# offset in y direction
        'θ':math.pi/2   ,# angle of rotation clockwise around origin
        'kx':0  ,# shearing parallel to x axis
        'ky':0  ,# shearing parallel to y axis 
        }
    
    rgb_profile = {
        'driver':'GTiff',
        "width": dim_x,
        "height": dim_y,
        "count": 3,
        "dtype": rgb.dtype,
        "crs": "EPSG:4326",
        "transform": calculate_affine(transform_coeffs),
        "nodata": np.nan,
        }
    
    fig_dir = "C://Users//Chris//co_cloudcover//reports//figures//"
    f_name = f"{rgb_bands['profile_name']}_{img_date}.tif"
    print(f"Writing {f_name}")
    rbgArray_to_RGBgeotiff(rgb,f_name,fig_dir,rgb_profile)

print(f"Done")

Writing snow_enhanced_2023-08-25.tif
Writing snow_enhanced_2023-08-07.tif
Writing snow_enhanced_2023-08-23.tif
Writing snow_enhanced_2023-08-28.tif
Writing snow_enhanced_2023-08-31.tif
Writing snow_enhanced_2023-08-12.tif
Writing snow_enhanced_2023-08-05.tif
Writing snow_enhanced_2023-08-03.tif
Writing snow_enhanced_2023-08-08.tif
Writing snow_enhanced_2023-08-10.tif
Writing snow_enhanced_2023-08-11.tif
Writing snow_enhanced_2023-08-13.tif
Writing snow_enhanced_2023-08-15.tif
Writing snow_enhanced_2023-08-17.tif
Writing snow_enhanced_2023-08-04.tif
Writing snow_enhanced_2023-08-01.tif
Writing snow_enhanced_2023-08-18.tif
Writing snow_enhanced_2023-08-02.tif
Writing snow_enhanced_2023-08-26.tif
Writing snow_enhanced_2023-08-21.tif
Writing snow_enhanced_2023-08-09.tif
Writing snow_enhanced_2023-08-06.tif
Writing snow_enhanced_2023-08-29.tif
Writing snow_enhanced_2023-08-27.tif
Writing snow_enhanced_2023-08-20.tif
Writing snow_enhanced_2023-08-24.tif
Writing snow_enhanced_2023-08-16.tif
W

#### 3. Write true color gtiffs
Standard true color images using band 1 (Red), band 4 (green) and band 3 (blue)

In [8]:
rgb_bands = {
    "profile_name" : "true_color",
    "red":"sur_refl_b01",
    "green":"sur_refl_b04",
    "blue":"sur_refl_b03",
    }

img_dates = list(set(IC_gdf["time"]))
for img_date in img_dates:
    img_date = img_date.strftime("%Y-%m-%d")
    
    rgb,dim_x,dim_y = make_RGBarray(IC_gdf,img_date,rgb_bands)
    
    transform_coeffs = {
        'sx':abs((min(IC_gdf['lon']) - max(IC_gdf['lon']))/dim_y)  ,# scale factor in x direction
        'sy':abs((min(IC_gdf['lat']) - max(IC_gdf['lat']))/dim_x)  ,# scale factor in y direction
        'tx':min(IC_gdf['lon'])  ,# offset in x direction
        'ty':min(IC_gdf['lat'])  ,# offset in y direction
        'θ':math.pi/2   ,# angle of rotation clockwise around origin
        'kx':0  ,# shearing parallel to x axis
        'ky':0  ,# shearing parallel to y axis 
        }
    
    rgb_profile = {
        'driver':'GTiff',
        "width": dim_x,
        "height": dim_y,
        "count": 3,
        "dtype": rgb.dtype,
        "crs": "EPSG:4326",
        "transform": calculate_affine(transform_coeffs),
        "nodata": np.nan,
        }
    
    fig_dir = "C://Users//Chris//co_cloudcover//reports//figures//"
    f_name = f"{rgb_bands['profile_name']}_{img_date}.tif"
    print(f"Writing {f_name}")
    rbgArray_to_RGBgeotiff(rgb,f_name,fig_dir,rgb_profile)

print(f"Done")

Writing true_color_2024-03-24.tif
Writing true_color_2024-03-01.tif
Writing true_color_2024-03-07.tif
Writing true_color_2024-03-18.tif
Writing true_color_2024-03-10.tif
Writing true_color_2024-03-03.tif
Writing true_color_2024-03-12.tif
Writing true_color_2024-03-08.tif
Writing true_color_2024-03-17.tif
Writing true_color_2024-03-04.tif
Writing true_color_2024-03-25.tif
Writing true_color_2024-03-05.tif
Writing true_color_2024-03-15.tif
Writing true_color_2024-03-19.tif
Writing true_color_2024-03-27.tif
Writing true_color_2024-03-09.tif
Writing true_color_2024-03-14.tif
Writing true_color_2024-03-30.tif
Writing true_color_2024-03-20.tif
Writing true_color_2024-03-02.tif
Writing true_color_2024-03-29.tif
Writing true_color_2024-03-22.tif
Writing true_color_2024-03-26.tif
Writing true_color_2024-03-28.tif
Writing true_color_2024-03-16.tif
Writing true_color_2024-03-13.tif
Writing true_color_2024-03-23.tif
Writing true_color_2024-03-31.tif
Writing true_color_2024-03-11.tif
Writing true_c

In [ ]:
sns.pairplot(IC_gdf)